In [22]:
import pandas as pd
from evaluate import load

In [61]:
wer = load("wer")

In [76]:
files = ['audios', 'gcp-stt', 'oci_speech', 'whisper-large-v3-turbo']
df_audios, df_stt, df_oci, df_whisper = (pd.read_csv(f"data/{f}.csv") for f in files)

In [84]:
for df in [df_audios, df_stt, df_oci, df_whisper]:
    display(df.head(3))

,path,sentence,duration,sr
0,data/audio/00010.mp3,Já é uma sorte que você é poupado de um acidente.,9.648,48000
1,data/audio/00076.mp3,"esquadrão da morte, jogo do bicho, máfia do ap...",9.756,48000
2,data/audio/00083.mp3,"burnout, anorexia nervosa, bulimia, periódica,...",9.972,48000


,file,transcription,total_seconds
0,00010.mp3,já é uma sorte que você é poupado de um acidente,7.97
1,00076.mp3,Esquadrão da Morte jogo do bicho máfia do apit...,9.19
2,00083.mp3,marxia nervosa bulimia perodica alimentar a co...,9.91


,file,transcription,total_seconds
0,00010.mp3,Já é uma sorte você é poupado de um acidente.,17
1,00076.mp3,"Esquadrão da Morte, Jogo do Bicho, Máfia do Ap...",17
2,00083.mp3,"Pernalte, anorexia nervosa, bulimia peróidica,...",21


,file,transcription,total_seconds
0,00010.mp3,já é uma sorte que você é poupado de um acidente,0.673280
1,00076.mp3,"Esquadrão da Morte, Jogo do Bicho, Máfia do A...",0.285897
2,00083.mp3,"...tornal, temorexia nervosa, bulimia periódi...",0.316535


In [188]:
def show_info(df_source, df_result):
    error = wer.compute(predictions=df_result.transcription, references=df_source.sentence)
    seconds = df_result.total_seconds.sum()
    minutes, seconds = divmod(int(seconds), 60)
    minutes_output = f"{minutes} minutes and {seconds} seconds"

    df_merge = (df_source
        .rename(columns={'path': 'file', 'sentence': 'reference'})
        .assign(file=df_audios.path.str.split('/').str[-1])
        .merge(df_result, on='file')
    )
    exec_per_second =  (df_merge.total_seconds / df_merge.duration).mean()

    print(f"Error (WER):               {error:.4f}")
    print(f"Mean exec. time by audio:  {df_result.total_seconds.mean():.2f}")
    print(f"Mean exec. per second:     {exec_per_second:.2f}")
    print(f"Total Time for 100 audios: {minutes_output}")

In [196]:
print('===== Google STT =====')
show_info(df_audios, df_stt)

===== Google STT =====
Error (WER):               0.4839
Mean exec. time by audio:  8.66
Mean exec. per second:     0.86
Total Time for 100 audios: 14 minutes and 26 seconds


In [197]:
print('===== OCI Speech =====')
show_info(df_audios, df_oci)

===== OCI Speech =====
Error (WER):               0.2638
Mean exec. time by audio:  22.01
Mean exec. per second:     2.19
Total Time for 100 audios: 36 minutes and 41 seconds


In [198]:
print('===== Whisper Large v2 Large Turbo + Flash Attn v2 =====')
show_info(df_audios, df_whisper)

===== Whisper Large v2 Large Turbo + Flash Attn v2 =====
Error (WER):               0.2381
Mean exec. time by audio:  0.24
Mean exec. per second:     0.02
Total Time for 100 audios: 0 minutes and 23 seconds
